# **Preprocessing Pokemon.csv File and creating Database**

In [16]:
import pandas as pd
import sqlite3
import json

pokemon_data = pd.read_csv("pokemon.csv",delimiter=',')
pokemon_data.head()

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,1,65,65,45,grass,poison,6.9,1,0
1,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,2,80,80,60,grass,poison,13.0,1,0
2,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,3,122,120,80,grass,poison,100.0,1,0
3,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,0
4,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,0


### **Ajout des noms Français**

In [17]:
json_file_path = 'pokemon.json'
with open(json_file_path, 'r', encoding='utf-8') as file:
    pokemon_json_data = json.load(file)

# Create a dictionary of English-to-French names from the JSON data
name_translation = {entry['name']['en']: entry['name']['fr'] for entry in pokemon_json_data if 'name' in entry}

# Add the `name_fr` column to the original CSV DataFrame
pokemon_data['name_fr'] = pokemon_data['name'].map(name_translation)

# Display a preview of the updated DataFrame
pokemon_data[['name', 'name_fr']].head()

,name,name_fr
0,Bulbasaur,Bulbizarre
1,Ivysaur,Herbizarre
2,Venusaur,Florizarre
3,Charmander,Salamèche
4,Charmeleon,Reptincel


### **Ajout du niveau d'évolution**

In [18]:
# Créer un dictionnaire associant les noms anglais des Pokémon à leur niveau d'évolution
evolution_levels = {}

for entry in pokemon_json_data:
    if 'name' in entry and isinstance(entry.get('evolution'), dict):
        name_en = entry['name']['en']
        evolution_data = entry['evolution']
        
        # Déterminer le niveau d'évolution
        if evolution_data.get('pre') is None and evolution_data.get('next') is not None:
            evolution_levels[name_en] = 1  # Pokémon de base
        elif evolution_data.get('pre') is not None and evolution_data.get('next') is not None:
            evolution_levels[name_en] = 2  # Pokémon intermédiaire
        elif evolution_data.get('next') is None:
            evolution_levels[name_en] = 3  # Pokémon final
    else:
        # Si le champ 'evolution' est manquant ou n'est pas un dictionnaire
        name_en = entry['name']['en']
        evolution_levels[name_en] = 1  # Niveau inconnu ou non applicable

# Ajouter la colonne 'Evolution' au DataFrame en mappant les noms anglais
pokemon_data['Evolution'] = pokemon_data['name'].map(evolution_levels)

### **Filtrage pour garder les bonnes colonnes**

In [19]:
# Liste des colonnes à conserver
colonnes_a_conserver = [
    'name', 'name_fr','Evolution','type1', 'type2',
    'weight_kg', 'height_m', 'generation',
    'is_legendary', 'pokedex_number', 'classfication'
]

# Création du nouveau DataFrame avec les colonnes sélectionnées
pokemon_selection = pokemon_data[colonnes_a_conserver]

# Affichage des premières lignes pour vérifier
pokemon_selection.head()

,name,name_fr,Evolution,type1,type2,weight_kg,height_m,generation,is_legendary,pokedex_number,classfication
0,Bulbasaur,Bulbizarre,1,grass,poison,6.9,0.7,1,0,1,Seed Pokémon
1,Ivysaur,Herbizarre,2,grass,poison,13.0,1.0,1,0,2,Seed Pokémon
2,Venusaur,Florizarre,3,grass,poison,100.0,2.0,1,0,3,Seed Pokémon
3,Charmander,Salamèche,1,fire,NaN,8.5,0.6,1,0,4,Lizard Pokémon
4,Charmeleon,Reptincel,2,fire,NaN,19.0,1.1,1,0,5,Flame Pokémon


### **Création de la base de donnée**

In [20]:
import sqlite3

# Connexion à la base de données SQLite (ou création si elle n'existe pas)
conn = sqlite3.connect('pokemon.db')
cursor = conn.cursor()

# Création de la table avec 'pokedex_number' comme clé primaire
cursor.execute('''
    CREATE TABLE IF NOT EXISTS pokemon (
        pokedex_number INTEGER PRIMARY KEY,
        name TEXT,
        name_fr TEXT,
        Evolution INTEGER,
        type1 TEXT,
        type2 TEXT,
        weight_kg REAL,
        height_m REAL,
        generation INTEGER,
        is_legendary INTEGER,
        classfication TEXT
    )
''')

# Création de la table player avec une clé primaire et unique id
cursor.execute('''
    CREATE TABLE IF NOT EXISTS player (
        discord_id TEXT PRIMARY KEY,
        player_name TEXT NOT NULL UNIQUE,
        win_count INTEGER DEFAULT NULL,
        win_streak INTEGER DEFAULT NULL
    )
''')

# Création de la table session avec des foreign keys
cursor.execute('''
    CREATE TABLE IF NOT EXISTS session (
        id INTEGER PRIMARY KEY,
        discord_id TEXT NOT NULL,
        pokedex_number INTEGER NOT NULL,
        found INTEGER DEFAULT NULL,
        name TEXT,
        name_fr TEXT,
        evolution INTEGER DEFAULT NULL,
        type1 TEXT,
        type2 TEXT DEFAULT NULL,
        weight_kg REAL,
        height_m REAL,
        generation INTEGER,
        is_legendary INTEGER,
        classfication TEXT,
        FOREIGN KEY (discord_id) REFERENCES player(discord_id) ON DELETE CASCADE,
        FOREIGN KEY (pokedex_number) REFERENCES pokemon(pokedex_number) ON DELETE CASCADE
    )
''')

# Insertion des données du DataFrame dans la table SQLite
pokemon_selection.to_sql('pokemon', conn, if_exists='replace', index=False)

# Fermeture de la connexion
conn.close()